# BeautifulSoup 으로 네이버 검색 결과 파싱하기


* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: requests, BeautifulSoup4

![search_blog_naver](images/search_blog_naver.jpg)


* 개발자도구로 대상 데이터의 DOM tree 형태를 확인하여 어떤 태그를 대상으로 할지 분석합니다.

![naver_address](images/naver_address.jpg)
* 네이버 검색시 주소줄에 어떤 주소로 쿼리가 이뤄지는지 확인해봐야 합니다.
* 쿼리는 클라이언트가 서버에게 어떤 데이터를 요청하는 질의어를 말합니다.
* URL 주소에 어떤 파라메타가 어떤 의미를 갖는지 대강 추측해봅니다.

In [ ]:
# 파싱을 위한 라이브러리
import requests
from bs4 import BeautifulSoup

# 검색어를 따로 관리
keyword = "파이썬강좌"

# 실제 검색이 이뤄지는 URL 주소와 쿼리문
url = "https://search.naver.com/search.naver?where=post&sm=tab_nmr&query={}&nso=".format(keyword)
r = requests.get(url)
bs = BeautifulSoup(r.text, "lxml")

for li in bs.select("li.sh_blog_top"):
    # 하위 태그로 다시 접근
    thumbnail = li.select("img.sh_blog_thumbnail")[0]["src"]
    title_link = li.select("dl > dt > a")[0]
    summary = li.select("dl > dd.sh_blog_passage")[0].text
    href = title_link["href"]
    title = title_link.text
    
    print((title, href, summary))
    


### 기능 확장하기
* 코드를 함수화 시켜 사용하기 좀 더 편리하게 만듭니다.
* 페이징 할 수 있게 함수의 기능을 확장합니다.

In [ ]:
'''
함수화 시키고 페이징 기능 추가
'''
import requests
from bs4 import BeautifulSoup

def get_search_naver_blog(keyword, start_page, end_page=None):
    '''네이버 블로그 검색 함수
    Args:
        keyword (str): 검색어
        start_page (int): 현재 페이지
        end_page (int): 마지막 페이지
        
    Returns:
        list : 최종 결과 목록
    '''
    
    # 검색 URL 주소
    # 네이버 페이징 처리는 1페이지, 2페이지 이런식이 아니라
    # startpage= 로 1, 11, 21 이런식으로 시작게시물 수의 개념으로 봐야함
    url = "https://search.naver.com/search.naver?where=post&sm=tab_nmr&query={}&nso=&start={}".format(keyword, start_page)

    # 네이버 서버에 URL 요청
    r = requests.get(url)
    # lxml 파서를 사용해서 응답내용 BeautifulSoup 으로 변환하여 bs 에 저장
    bs = BeautifulSoup(r.text, "lxml")
    
    # 최종 결과를 리턴할 리스트
    results = []

    # 최초 1페이지시에는 end_page 값이 None으로 넘어오기 때문에 이때 end_page를 구해야함
    if end_page is None:
        # 총 게시물 갯수는 이렇게 표기되므로 여기서 필요한 값만 파싱
        # <span class="title_num">1-10 / 6,283건</span> 에서
        # '/' 문자를 기준으로 split 하여 뒷쪽 값을 취함[-1]
        # 뒷쪽 값 6,284건 에서 '건' 글자와 ',' 글자를 없애서 tot_count 변수에 int 형으로 저장
        tot_count = int(bs.select("span.title_num")[0].text.split("/")[-1].replace("건", "").replace(",", ""))
        # 총게시물 갯수 / 10 을 int 형으로 하면 end_page 값을 알 수 있음
        end_page = int(tot_count / 10)
        
        # end_page 가 50이 넘어가면 그냥 51까지만 하기로 함
        # 어차피 뒷쪽 데이터는 오래된 게시물 혹은 정확도가 떨어지는 데이터로 판단하고
        # 적절한 페이지 수에서 타협
        if end_page > 50:
            end_page = 51

    # 블로그 검색 목록만큼 반복문
    for li in bs.select("li.sh_blog_top"):
        # 결과 파싱 시에 오류가 날 수 있으므로
        try:
            # 하위 태그로 다시 접근 하여 원하는 데이터를 수집
            thumbnail = li.select("img.sh_blog_thumbnail")[0]["src"]
            title_link = li.select("dl > dt > a")[0]
            summary = li.select("dl > dd.sh_blog_passage")[0].text
            href = title_link["href"]
            title = title_link.text

            # 최종 결과 리스트에 튜플 형태로 append()
            results.append((title, href, summary, thumbnail))
        except:
            # 오류 발생시 다음 반복문을 수행
            continue

    # 현재 페이지가 마지막 페이지보다 작으면 
    if start_page < end_page:
        # 현재 페이지에 + 10증가 후 
        start_page += 10
        # 함수 스스로 재호출(재귀함수) 후 결과를 현재 results(결과를 리턴할 리스트)에 확장(extend) 시킴
        results.extend(get_search_naver_blog(keyword, start_page, end_page))
    
    # 최종 결과 리스트 리턴
    return results


# 함수 호출
results = get_search_naver_blog("파이썬강좌", 1)

# 결과 출력
for r in results:
    print(r)